In [3]:
import nbimporter
from RADARDataCapture import FMCWRADARDataCapture
import h5py
import numpy as np
import imageio
import matplotlib.pyplot as plt

In [2]:
test_path="/Users/danielcopeland/Library/CloudStorage/OneDrive-MassachusettsInstituteofTechnology/Yoga_Study_RADAR_Data/02/02_CRW2L_RR_V1.h5"

test_data = FMCWRADARDataCapture(file_path=test_path)
test_data.load_and_save()

(1000, 128, 256, 4)
